In [1]:
spark

In [98]:
dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2a.problematic_members_excluded/'
file_flag = 'msno_a-n'

In [3]:
infile = dir0 + file_flag
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- tid: string (nullable = true)
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [10]:
df0.limit(5).toPandas()

,tid,msno,pay_method,plan_days,list_price,actual_paid,auto_renew,is_cancel,trans_date,expire_date,corr_exp_date,corr_start_date
0,17179869188,eyxUoQJ7Jb3h2JRBm9P1+6agI2TrGBRWLwE9Fhu+oMg=,41,30,149,149,true,false,2017-01-31,2017-03-29,2017-03-29,2017-02-28
1,17179869193,dBYBzgvGgqYNI3q4SUV70joE9RPc/vStbbbLYO2nEEY=,40,30,149,149,true,false,2016-11-14,2016-12-16,2016-12-16,2016-11-17
2,17179869194,evstAjTz7qlRoRjzS8MEJciKm2dR6c5vB93IwG9BQ7I=,40,30,149,149,true,false,2016-09-10,2016-10-09,2016-10-09,2016-09-10
3,17179869196,aTeoZgUBG4R/Cq8ewJ+apYHDYqlmyMXeUGcPCBZCp2s=,41,30,129,129,true,false,2016-05-10,2016-06-10,2016-06-10,2016-05-12
4,17179869211,nBkJ6ORyG8iptHWFrP4/4e4AVkIWnVCklerE8JpQQK8=,39,30,149,149,true,false,2016-07-31,2016-09-20,2016-09-20,2016-08-22


In [5]:
from pyspark.sql.functions import col
df0.where(col('is_cancel') == 'true').show(1)

+-----------+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|        tid|                msno|pay_method|plan_days|list_price|actual_paid|auto_renew|is_cancel|trans_date|expire_date|corr_exp_date|corr_start_date|
+-----------+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|17179869219|aRdKXBwEHOUprPT90...|        41|       30|       149|        149|      true|     true|2016-02-25| 2016-02-25|   2016-02-25|           null|
+-----------+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
only showing top 1 row



# Collect Infos for Each Member 

In [6]:
columns = ('tid', 'trans_date', 'is_cancel', 'corr_start_date', 'corr_exp_date')
df1 = df0.select('msno', *columns)
df1.show(5)

+--------------------+-----------+----------+---------+---------------+-------------+
|                msno|        tid|trans_date|is_cancel|corr_start_date|corr_exp_date|
+--------------------+-----------+----------+---------+---------------+-------------+
|eyxUoQJ7Jb3h2JRBm...|17179869188|2017-01-31|    false|     2017-02-28|   2017-03-29|
|dBYBzgvGgqYNI3q4S...|17179869193|2016-11-14|    false|     2016-11-17|   2016-12-16|
|evstAjTz7qlRoRjzS...|17179869194|2016-09-10|    false|     2016-09-10|   2016-10-09|
|aTeoZgUBG4R/Cq8ew...|17179869196|2016-05-10|    false|     2016-05-12|   2016-06-10|
|nBkJ6ORyG8iptHWFr...|17179869211|2016-07-31|    false|     2016-08-22|   2016-09-20|
+--------------------+-----------+----------+---------+---------------+-------------+
only showing top 5 rows



In [14]:
from pyspark.sql.functions import concat_ws
df2 = df1.fillna('na', ['corr_start_date']) \
      .select('msno',
              concat_ws('/', *columns).alias('info'))

df2.show(5, truncate=False)

+--------------------------------------------+--------------------------------------------------+
|msno                                        |info                                              |
+--------------------------------------------+--------------------------------------------------+
|eyxUoQJ7Jb3h2JRBm9P1+6agI2TrGBRWLwE9Fhu+oMg=|17179869188/2017-01-31/false/2017-02-28/2017-03-29|
|dBYBzgvGgqYNI3q4SUV70joE9RPc/vStbbbLYO2nEEY=|17179869193/2016-11-14/false/2016-11-17/2016-12-16|
|evstAjTz7qlRoRjzS8MEJciKm2dR6c5vB93IwG9BQ7I=|17179869194/2016-09-10/false/2016-09-10/2016-10-09|
|aTeoZgUBG4R/Cq8ewJ+apYHDYqlmyMXeUGcPCBZCp2s=|17179869196/2016-05-10/false/2016-05-12/2016-06-10|
|nBkJ6ORyG8iptHWFrP4/4e4AVkIWnVCklerE8JpQQK8=|17179869211/2016-07-31/false/2016-08-22/2016-09-20|
+--------------------------------------------+--------------------------------------------------+
only showing top 5 rows



In [17]:
from pyspark.sql.functions import concat_ws, collect_list
df3 = df2.groupBy('msno').agg(concat_ws('|', collect_list('info')).alias('infos'))

df3.show(1)

+--------------------+--------------------+
|                msno|               infos|
+--------------------+--------------------+
|a+J+UfAgFv9t5OJtj...|25770307879/2016-...|
+--------------------+--------------------+
only showing top 1 row



In [18]:
# output (temp)
outfile = dir0 + 'temp'
df3.write.format('csv').option('header','true').save(outfile)

# Reduce Transaction Records to Effective Subscriptions

In [85]:
# input (temp)
infile = dir0 + 'temp'
df0 = spark.read.format('csv').option('header','true').load(infile)
rdd0 = df0.rdd

In [7]:
rdd0.take(4)

[Row(msno='a+J6FfukEkxO6rwVx2waiL1m9XNLoEVydLw0z33G2Q0=', infos='25769987128/2016-10-31/false/2016-11-25/2016-12-24|25770444358/2015-12-02/false/2015-12-26/2016-01-24|68719763999/2016-01-31/false/2016-02-24/2016-03-24|103079869450/2016-08-31/false/2016-09-25/2016-10-24|120259379193/2016-12-31/false/2017-01-26/2017-02-24|111669210736/2017-01-31/false/2017-02-23/2017-03-24|128849651595/2015-11-02/false/2015-11-25/2015-12-24|137438996617/2016-09-30/false/2016-10-26/2016-11-24|17180500924/2016-07-31/false/2016-08-26/2016-09-24|34360074228/2016-11-30/false/2016-12-26/2017-01-24'),
 Row(msno='a+ldj/Z4NQLXp1dhKwcmdVvZpTz9T579jm+kcdD1IlQ=', infos='68719970166/2016-12-04/false/2016-12-05/2017-01-03|8589966754/2016-07-22/false/2016-07-23/2016-08-21|272310/2016-06-21/false/2016-06-23/2016-07-22|128849615257/2016-05-23/false/2016-05-24/2016-06-22|111669328183/2016-03-25/false/2016-03-26/2016-04-24|17180675772/2016-08-19/false/2016-08-22/2016-09-20'),
 Row(msno='a+vUjyTIKus85eXa/suo1jzFeIGE2rEBp1ma

In [111]:
import datetime as DT

def add_one_day(date_str):
    t = DT.datetime.strptime(date_str, '%Y-%m-%d') + DT.timedelta(days=1)
    return t.strftime('%Y-%m-%d')


def subtract_one_day(date_str):
    t = DT.datetime.strptime(date_str, '%Y-%m-%d') - DT.timedelta(days=1)
    return t.strftime('%Y-%m-%d')


def find_indeterminate_period(tinfo1, tinfo2):
    tinfo_cancel, tinfo_noncancel = (tinfo1, tinfo2)  if tinfo1[2] == 'true'  else (tinfo2, tinfo1)
    dc = tinfo_cancel[4]
    if dc >= tinfo_noncancel[4]:
        return None
    elif dc >= tinfo_noncancel[3]:
        return (add_one_day(dc), tinfo_noncancel[4])
    else: # dc < tinfo_noncancel[3]
        return (tinfo_noncancel[3], tinfo_noncancel[4])

    
def reduce_subscription_periods(subscrip_periods, cut_date):
    '''
    cut all periods in `subscrip_periods` to before `cut_date`
    '''
    subscrip_periods_copy = subscrip_periods[:]
    pre_cut_date = subtract_one_day(cut_date)
    
    for p in subscrip_periods_copy:
        if cut_date <= p[1] :
            subscrip_periods.remove(p)
            
    for i in range(0, len(subscrip_periods)):
        p = subscrip_periods[i]
        if cut_date <= p[2]:
            subscrip_periods[i] = (p[0], p[1], pre_cut_date)
            

def has_no_cancel(tinfo_records):
    r = True
    for is_cancel in map(lambda info_rec: info_rec[2]=='true', tinfo_records):
        if is_cancel:
            r = False
            break
    return r


def reduce_trans_infos(trans_infos_str):
    '''
    Deals with cancels.
    Determines effective subscription periods.
    Returns (subscription-periods, indeterminate-periods).
    '''
    trans_infos_strlist = trans_infos_str.split('|')
    trans_infos = [s.split('/')  for s in trans_infos_strlist]
        #
        # trans_infos[trans-index][item-index]  
        # item-index  --  0: tid;  1: transaction date;  2: is cancel;  
        #                 3: start date;  4: end (expiration) date
    
    ##
    if has_no_cancel(trans_infos):
        subscrip_periods = [(tinfo[0], tinfo[3], tinfo[4])  for tinfo in trans_infos]
        return (subscrip_periods, [])
            
    ## separate the transaction records by transaction dates having a cancel
    ### get transaction dates having a cancel
    trans_dates_w_cancel_s = set()
    last_trans_date = '1970-01-01'
    for tinfos in trans_infos:
        if tinfos[2] == 'true':
            trans_dates_w_cancel_s.add(tinfos[1])
        if tinfos[1] > last_trans_date:
            last_trans_date = tinfos[1]
        
    separating_dates = sorted(list(trans_dates_w_cancel_s)) + [last_trans_date]

    ###
    trans_info_groups = []
    for sdate in separating_dates:
        group = [tinfos  for tinfos in trans_infos  if tinfos[1] <= sdate]
        if group != []:
            trans_info_groups.append(group)
            for info in group:
                trans_infos.remove(info)
        # now `trans_infos` is destroyed
        # use `trans_info_groups` instead
        
    ## deal with each group
    subscrip_periods = []
    indeterm_periods = []
    for tinfos in trans_info_groups:
        assert tinfos != []
        
        ## sort by transaction date
        tinfos.sort(key=(lambda tinfo: tinfo[1]))
                
        ##
        if has_no_cancel(tinfos):
            subscrip_periods_1 = [(tinfo[0], tinfo[3], tinfo[4])  for tinfo in tinfos]
            subscrip_periods.extend(subscrip_periods_1)
            continue
            
        ## get transaction infos before last transaction date
        last_tdate = tinfos[-1][1]
        
        ind_pre_last_date = -1
        for i in range(len(tinfos)-1, -1, -1):
            if tinfos[i][1] != last_tdate:
                ind_pre_last_date = i
                break
        tinfos_pre_last_date = tinfos[0:ind_pre_last_date+1]  
        assert (ind_pre_last_date+1) < len(tinfos)
        
        ## transform `tinfos_pre_last_date` into subscription periods
        subscrip_periods_1 = [(tinfo[0], tinfo[3], tinfo[4])  for tinfo in tinfos_pre_last_date]
            
        ## find whether there is non-cancel transaction on the last transaction date
        has_non_cancel = any([tinfo[2]=='false'  for tinfo in tinfos[ind_pre_last_date+1:]])
        
        ##
        indet_period = None
        if has_non_cancel:  # one cancel and one non-cancel on `last_tdate`
            indet_period = find_indeterminate_period(tinfos[-2], tinfos[-1])
            if indet_period is not None:
                cut_date = indet_period[0]
                reduce_subscription_periods(subscrip_periods_1, cut_date)
        else:  # only cancels on `last_tdate`
            cut_date = min([tinfo[4] for tinfo in tinfos[ind_pre_last_date+1:]])
            reduce_subscription_periods(subscrip_periods_1, cut_date)
        
        ## store results
        subscrip_periods.extend(subscrip_periods_1)
        if indet_period is not None:
            indeterm_periods.append(indet_period)
    ## END of looping trans_info_groups
    
    return (subscrip_periods, indeterm_periods)


In [103]:
trans_infos = '51539718401/2016-05-05/false/2016-06-08/2016-07-07|51540051214/2016-07-05/false/2016-08-08/2016-09-06|' \
  + '51540124967/2016-09-05/false/2016-10-08/2016-11-06|51540369529/2016-01-24/false/2016-02-27/2016-03-27|' \
  + '8589999995/2016-11-05/false/2016-12-08/2017-01-06|94489594123/2016-02-06/true/na/2016-03-08|' \
  + '94490024042/2017-02-05/false/2017-02-05/2017-03-06|146029241083/2015-12-24/false/2016-01-27/2016-02-25|' \
  + '103079250926/2016-04-05/false/2016-05-08/2016-06-06|103079361450/2016-03-05/false/2016-04-08/2016-05-07|' \
  + '111669172613/2016-02-06/false/2016-03-08/2016-04-06|154619145884/2017-01-05/false/2017-01-08/2017-02-06|' \
  + '154619268354/2016-08-05/false/2016-09-08/2016-10-07|17180245295/2015-11-24/false/2015-12-27/2016-01-25|' \
  + '60129868029/2016-10-05/false/2016-11-08/2016-12-07|85899981601/2016-06-05/false/2016-07-08/2016-08-06'
reduce_trans_infos(trans_infos)

([('17180245295', '2015-12-27', '2016-01-25'),
  ('146029241083', '2016-01-27', '2016-02-25'),
  ('51540369529', '2016-02-27', '2016-03-08'),
  ('103079361450', '2016-04-08', '2016-05-07'),
  ('103079250926', '2016-05-08', '2016-06-06'),
  ('51539718401', '2016-06-08', '2016-07-07'),
  ('85899981601', '2016-07-08', '2016-08-06'),
  ('51540051214', '2016-08-08', '2016-09-06'),
  ('154619268354', '2016-09-08', '2016-10-07'),
  ('51540124967', '2016-10-08', '2016-11-06'),
  ('60129868029', '2016-11-08', '2016-12-07'),
  ('8589999995', '2016-12-08', '2017-01-06'),
  ('154619145884', '2017-01-08', '2017-02-06'),
  ('94490024042', '2017-02-05', '2017-03-06')],
 [('2016-03-09', '2016-04-06')])

In [91]:
def pack_data(msno, subscrip_periods, indeterm_periods):
    return [(msno, it[0], it[1], it[2])  for it in subscrip_periods] \
            + [(msno, 'indeterm', it[0], it[1])  for it in indeterm_periods]

pack_data('--msno--', *reduce_trans_infos(trans_infos))

[('--msno--', '17180245295', '2015-12-27', '2016-01-25'),
 ('--msno--', '146029241083', '2016-01-27', '2016-02-25'),
 ('--msno--', '51540369529', '2016-02-27', '2016-03-08'),
 ('--msno--', '103079361450', '2016-04-08', '2016-05-07'),
 ('--msno--', '103079250926', '2016-05-08', '2016-06-06'),
 ('--msno--', '51539718401', '2016-06-08', '2016-07-07'),
 ('--msno--', '85899981601', '2016-07-08', '2016-08-06'),
 ('--msno--', '51540051214', '2016-08-08', '2016-09-06'),
 ('--msno--', '154619268354', '2016-09-08', '2016-10-07'),
 ('--msno--', '51540124967', '2016-10-08', '2016-11-06'),
 ('--msno--', '60129868029', '2016-11-08', '2016-12-07'),
 ('--msno--', '8589999995', '2016-12-08', '2017-01-06'),
 ('--msno--', '154619145884', '2017-01-08', '2017-02-06'),
 ('--msno--', '94490024042', '2017-02-05', '2017-03-06'),
 ('--msno--', 'indeterm', '2016-03-09', '2016-04-06')]

In [115]:
rdd1 = rdd0.flatMap(lambda row: pack_data(row['msno'], *reduce_trans_infos(row['infos'])))
rdd2 = rdd1.map(lambda rec: ','.join(rec))

rdd2.take(3)

['a+J6FfukEkxO6rwVx2waiL1m9XNLoEVydLw0z33G2Q0=,25769987128,2016-11-25,2016-12-24',
 'a+J6FfukEkxO6rwVx2waiL1m9XNLoEVydLw0z33G2Q0=,25770444358,2015-12-26,2016-01-24',
 'a+J6FfukEkxO6rwVx2waiL1m9XNLoEVydLw0z33G2Q0=,68719763999,2016-02-24,2016-03-24']

In [116]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/3.subscription_periods/' \
          + file_flag
rdd2.saveAsTextFile(outfile)